# Test image dimensions

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from plantclef.utils import get_spark
from pyspark.sql import functions as F

spark = get_spark()
display(spark)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/05 21:36:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/05 21:36:46 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [3]:
# Path and dataset names
gcs_path = "gs://dsgt-clef-plantclef-2024/data"
test_path = "parquet_files/PlantCLEF2024_test"

# Read the Parquet file into a DataFrame
test_df = spark.read.parquet(f"{gcs_path}/{test_path}")
test_df.show(n=5, truncate=50)

+-------------------------------------------+------------------------+--------------------------------------------------+
|                                       path|              image_name|                                              data|
+-------------------------------------------+------------------------+--------------------------------------------------+
| /PlantCLEF2024test/CBN-Pla-B4-20160728.jpg| CBN-Pla-B4-20160728.jpg|[FF D8 FF E0 00 10 4A 46 49 46 00 01 01 01 00 4...|
| /PlantCLEF2024test/CBN-Pla-D3-20130808.jpg| CBN-Pla-D3-20130808.jpg|[FF D8 FF E0 00 10 4A 46 49 46 00 01 01 01 00 4...|
|/PlantCLEF2024test/CBN-PdlC-E4-20150701.jpg|CBN-PdlC-E4-20150701.jpg|[FF D8 FF E0 00 10 4A 46 49 46 00 01 01 01 00 4...|
| /PlantCLEF2024test/CBN-Pla-F5-20150901.jpg| CBN-Pla-F5-20150901.jpg|[FF D8 FF E0 00 10 4A 46 49 46 00 01 01 01 00 4...|
| /PlantCLEF2024test/CBN-Pla-D1-20180724.jpg| CBN-Pla-D1-20180724.jpg|[FF D8 FF E0 00 10 4A 46 49 46 00 01 01 01 00 4...|
+-----------------------

In [4]:
test_df.printSchema()

root
 |-- path: string (nullable = true)
 |-- image_name: string (nullable = true)
 |-- data: binary (nullable = true)



In [5]:
from PIL import Image
import io


# get image at specific index
def get_image_dimensions(df, index: int):
    # Add an index column to the DataFrame
    df_with_index = df.withColumn("index", F.monotonically_increasing_id())
    # Extract the binary data for the image at the specified index
    image_data = (
        df_with_index.filter(df_with_index.index == index).select("data").first()[0]
    )
    # Convert the binary data to an image
    image = Image.open(io.BytesIO(image_data))
    return image.size


# print dimensions
for i in range(10):
    image_dim = get_image_dimensions(test_df, index=i)
    print(f"Image {i} dimensions: {image_dim}")

Image 0 dimensions: (3068, 2884)
Image 1 dimensions: (3000, 3156)
Image 2 dimensions: (3112, 3088)
Image 3 dimensions: (3100, 2848)
Image 4 dimensions: (3044, 2964)
Image 5 dimensions: (2956, 2600)
Image 6 dimensions: (2920, 2848)
Image 7 dimensions: (3236, 2728)
Image 8 dimensions: (3024, 3024)
Image 9 dimensions: (2464, 2420)
